In [2]:
import pandas as pd
# from google.colab import drive

**LOAD DATA**

In [ ]:
#
drive.mount('/content/drive', force_remount=True)
df = pd.read_csv('/content/drive/MyDrive/STAT 390/convert_MCMF_ALL_TIME_DATA.csv', sep = '\t')

In [ ]:
# load population data (used to calculate per capita metrics)
population_df = pd.read_csv('/content/drive/MyDrive/STAT 390/chicago_population.CSV', ';').T
headers = population_df.iloc[0]
population_df = pd.DataFrame(population_df.values[1:], columns=headers)

In [ ]:
temp1 = df['Geographic Cluster Name'].sort_values().unique()
temp1

In [ ]:
temp2 = population_df['name'].str.upper().str.replace("'", "").sort_values().unique()
temp2

In [ ]:
# Explore differences in community areas before merging population data and main data
set(temp1) ^ set(temp2)

**PREPROCESSING**

In [ ]:
# Discard programs with min age over 25
df = df.loc[df['Min Age'] <= 25]

# Record programs with max age over 25 as family programs
df['Family Program'] = (df['Max Age'] > 25).astype(bool)

# Make category names unique
category_dict = {'Building And Fixing Things': 'Building & Fixing Things', 
 'Computers.': 'Computers', 
 'Food.': 'Food',
 'Helping Your Community.': 'Helping Your Community',
 'Reading & Writing.': 'Reading & Writing',
 'Sports + Wellness.': 'Sports & Wellness',
 'Work + Career': 'Work & Career'
}

df['Category Name'] = df['Category Name'].replace(category_dict)

# Group category names by STEM
stem_categories = ['Computers', 'Science', 'Building & Fixing Things', 'Science & Math', 'Digital Media.', 'Nature.', 'Math']
df['STEM'] = df['Category Name'].isin(stem_categories)
df['STEM'] = df['STEM'].map({True: 'STEM', False: 'Non-STEM'})

# Group min and max age by quartile
df['Min Age Quartile'] = pd.cut(df['Min Age'], bins=[0, 10, 17, 99, 200], labels=['0-10', '11-17','18-99', '100-200'])
df['Max Age Quartile'] = pd.cut(df['Max Age'], bins=[0, 3, 6, 13, 25], labels=['0-3', '4-6', '7-13','14-25'])

**ANALYSIS**

In [ ]:
# Exploring null values
df.isnull().sum()

# Examining the two observations without category names
df.loc[df['Program Name'] == 'Grab & Go Meals for Children & Teens']
df.loc[df['Program Name'] == 'Trunk or Treat at Shabbona']

# Examining observations with missing Geographic cluster names
geonan = df.loc[df['Geographic Cluster Name'].isnull()]
geonan.isnull().sum()

# Checking observations without geographic cluster name and no online address
geoonlinenan = geonan.loc[geonan['Online Address'].isnull()].groupby(by = 'Category Name')
num = geoonlinenan['Category Name'].count()

# Checking the actual category names
categories = df.groupby(by = 'Category Name')
categories['Category Name'].count()

# Locating observations with min_age = 25
df['Min Age'].dtype
df.loc[df['Min Age'] == 25]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

clusters = df.groupby('Geographic Cluster Name').agg({'Geographic Cluster Name': 'count', 'Capacity':'sum', 'Max Age':'mean', 'Latitude':'mean'})
clusters.columns = ['Total Programs', 'Total Capacity', 'Average Max Age', 'Average Latitude']
clusters['Average Capacity'] = clusters['Total Capacity'] / clusters['Total Programs']
clusters.sort_values('Total Programs', inplace=True, ascending=False)
clusters.reset_index(inplace=True)
print(clusters)

# Train on total capacity for different program types?
    # Cluster, Type, 
    # Make a binary column for online/in-person
    
# Add population for per capita

clusters.plot.bar(x='Geographic Cluster Name', y='Total Programs')
clusters.sort_values('Total Capacity', ascending=False).plot.bar(x='Geographic Cluster Name', y='Total Capacity')
clusters.plot(kind='scatter', x='Average Latitude', y='Total Programs')

# Train on total capacity for different program types

    ## Important Predictors- what to group by?
    # Cluster (location)
    # Category (make subcategories?)
    # Binary online/in-person
    
    ## Other features to create/use?
    # Time of the year?
    # Price?
    
    ## 
    # Other predictors?
    # Other metrics/interactions to analyze?

grouped = clusters = df.groupby(['Geographic Cluster Name', 'Category Name', 'Meeting Type']).agg({'Geographic Cluster Name': 'count', 'Capacity':'sum', 'Max Age':'mean', 'Latitude':'mean'})
grouped.columns = ['Total Programs', 'Total Capacity', 'Average Max Age', 'Average Latitude']
grouped['Average Capacity'] = grouped['Total Capacity'] / grouped['Total Programs']


grouped.reset_index(inplace=True)
print(grouped)

# This grouped dataframe will evolve with new enigneered predictors and metrics. We will use this as out training data to assess equity moving forward


### Equity Metric

In [ ]:
## Equity index - how do we model areas of need
# 2016-2020 Chicago Community Hardship Index : Percent Values 2016-2020
hardship = pd.read_csv('Hardship_Index.csv')

# Hardship Score - can talk about how this is calculated during presentation
# https://storymaps.arcgis.com/stories/da5601c3e0924e5ab3ee07ade9954f7a#

In [ ]:
hardship['Community Area'] = hardship['Community Area'].str.upper()
merge = pd.merge(grouped, hardship, left_on='Geographic Cluster Name', right_on='Community Area')


In [3]:
grouped[~grouped['Geographic Cluster Name'].isin(hardship['Community Area'])]['Geographic Cluster Name'].unique()

# Need to do some manual matching to resolve merge issues with hardship and popualtion data

NameError: name 'grouped' is not defined